In [121]:
import mhdb
import pandas as pd

In [125]:
infile = "/Users/jon.clucas/Downloads/core neutral behaviors (3).csv"
outfile = "/Users/jon.clucas/Downloads/core neutral behaviors.ttl"
chart = pd.read_csv(infile, index_col="Id")
chart = chart.loc[chart["Name"].isin(["Entity", "Attribute", "Line"])]

In [126]:
def rdfstring(label, superclasses, entity_type):
    
    rdf_string = """
### {0}
:{1} rdf:type :{3} ;
    rdfs:label \"{2}\" """.format(
        label,
        mhdb.spreadsheet_io.convert_string_to_label(label),
        label,
        "owl:Class"
    )
    for superclass in superclasses:
        rdf_string += """;
    rdfs:subClassOf :{0} """.format(superclass)
    return("{0} .".format(rdf_string))

In [127]:
rdf = mhdb.write_rdf.print_header(
    "http://mentalhealth.tech/core",
    "0.0.2",
    "neutral behaviors",
    "core neutral behaviors")

for row in chart.iterrows():
    if(row[1].Name in ["Entity", "Attribute"]):
        supers = ([*[mhdb.spreadsheet_io.convert_string_to_label(
            chart.loc[chart.index == x]["Text Area 1"].values[0]) for x in chart.loc[
            (chart["Line Source"] == row[1].name)
             &
             (chart["Source Arrow"] == "Arrow")
        ]["Line Destination"]], *[mhdb.spreadsheet_io.convert_string_to_label(
            chart.loc[chart.index == x]["Text Area 1"].values[0]) for x in chart.loc[
            (chart["Line Destination"] == row[1].name)
             &
             (chart["Destination Arrow"] == "Arrow")
        ]["Line Source"]]])
        rdf += rdfstring(row[1]["Text Area 1"], supers, row[1].Name)
        
with open(outfile, "w") as o:
    o.write(rdf)